# The Cornology Case - Optimization for Market Entry
**A2 Data Analysis Team Project**

**Hult International Business School** 

**2022 Spring Team 1 Members**

Manuel Echazarra Casado

Yuan Ning Chiang 

Tien-Wei Hsu 

Sicong Li 

Timothy Naman 

Linda Webb



## Import Libraries and Relevant Data

In [1]:
# Import Libraries required

import pulp as p
import pandas as pd
import numpy as np

# Set the profits variables in an array
profit_matrix = np.array([12500,7500]).reshape(2,1)

## Define production maximums for orignal and flavor blast

# One unit orginal needs 15,000 seeds, and 12,500 bottles 
# There is an overall limit on seeds and bottles that can be produced
# This results in an effective maximum units that can be produced in a production run
og_pr_max = min(1350000/15000,1575000/12500)

# One unit flavor blast needs 12,500 seeds, and 22,500 bottles
fb_pr_max = min(1350000/12500,1575000/22500)

#printing to confirm the variables are correct
print(f"""The maximum original unit is {og_pr_max} per product run.""")
print(f"""The maximum flavor blast unit is {fb_pr_max} per product run.""")

# Use np.array to get the maximum production amounts into one array
pr_max = np.array([og_pr_max, fb_pr_max]).reshape(2, 1)

# Input the demand for original by region and flavor blast per region
#              | North East | YRD | South East | North Central | South Central| Western China
# --------------------------------------------------------------------------------------------
# Original     |
# Flavor Blast |

dmd_matrix = np.array([[0,180,127,90,118,90],[70,112,37,70,74,0]])

# Create an adjusted demand matrix
# Accounts for only needing to satisfy 90 percent of a region's demand
min_pr_matrix = np.array(np.ceil(dmd_matrix*0.9/pr_max))

# Create a matrix with the maximum that can be produced
# Based on the number of production runs and the maximum number of units per run
min_dmd_matrix = np.multiply(min_pr_matrix,pr_max)

# The final demand matrix which is the minimum of the actual demand and the maximum that can be produced
min_sup_matrix = np.minimum(dmd_matrix,min_dmd_matrix)

print(f"""The demand matrix by region and flavor type: 
{dmd_matrix}""")

print(f"""The production runs per region and flavor type matrix: 
{min_pr_matrix}""")

print(f"""The demand that can be met based on number of production runs: 
{min_dmd_matrix}""")

print(f"""The final demand that will be met subject to the limits: 
{min_sup_matrix}""")

The maximum original unit is 90.0 per product run.
The maximum flavor blast unit is 70.0 per product run.
The demand matrix by region and flavor type: 
[[  0 180 127  90 118  90]
 [ 70 112  37  70  74   0]]
The production runs per region and flavor type matrix: 
[[0. 2. 2. 1. 2. 1.]
 [1. 2. 1. 1. 1. 0.]]
The demand that can be met based on number of production runs: 
[[  0. 180. 180.  90. 180.  90.]
 [ 70. 140.  70.  70.  70.   0.]]
The final demand that will be met subject to the limits: 
[[  0. 180. 127.  90. 118.  90.]
 [ 70. 112.  37.  70.  70.   0.]]


## Define LP Problem and Decision Variables

In [2]:
# Define the LP problem and confirm it is a maximization problem

model = p.LpProblem(name = "Cornology-Problem", sense = p.LpMaximize)

# Define the number of regions
n_regions = 6

# Create the string of the six regions
# 1 - North East
# 2 - YRD
# 3 - South East
# 4 - North Central
# 5 - South Central
# 6 - Western China
variable_names = [str(i) for i in range(1, n_regions+1)]

# Sort the names in correct order
variable_names.sort(reverse = False)

# Use the number string to add decision varibles X_1 to X6 to the model
# The model has binary variables, 1 if we move into the region, 0 if not
DV_variables = p.LpVariable.matrix("X", variable_names, 
                                 cat      = "Binary", 
                                 lowBound = 0,
                                 upBound  = 0)

# Reshaping the decision variables
allocation = np.array(DV_variables).reshape(1,6)

# Print them out to make sure it is working
print(f"""Decision Variable/Allocation Matrix: 
{allocation}""")

Decision Variable/Allocation Matrix: 
[[X_1 X_2 X_3 X_4 X_5 X_6]]


In [3]:
# Multiply out the final demand number by the variables
sup_matrix = np.multiply(allocation, min_sup_matrix, out = None)

print(f"""The final demand with variable matrix is:
{sup_matrix}""")

The final demand with variable matrix is:
[[0 180.0*X_2 + 0.0 127.0*X_3 + 0.0 90.0*X_4 + 0.0 118.0*X_5 + 0.0
  90.0*X_6 + 0.0]
 [70.0*X_1 + 0.0 112.0*X_2 + 0.0 37.0*X_3 + 0.0 70.0*X_4 + 0.0
  70.0*X_5 + 0.0 0]]


## Define Objective Function

In [4]:
# The objective function only takes into account the expected profit and the demand that can be met
# Any once off set up costs are not included
obj_func = p.lpSum(profit_matrix * sup_matrix)

# Add the objective funtion to the model
model +=  obj_func

# Print out the model to ensure it is specified correctly
print(model)


Cornology-Problem:
MAXIMIZE
525000.0*X_1 + 3090000.0*X_2 + 1865000.0*X_3 + 1650000.0*X_4 + 2000000.0*X_5 + 1125000.0*X_6 + 0.0
VARIABLES
0 <= X_1 <= 1 Integer
0 <= X_2 <= 1 Integer
0 <= X_3 <= 1 Integer
0 <= X_4 <= 1 Integer
0 <= X_5 <= 1 Integer
0 <= X_6 <= 1 Integer



## Define Constraints

In [5]:
# For one of the constraints we need a matrix with the decision variable multiplied by the number of production runs
pr_matrix = np.multiply(allocation, min_pr_matrix)

# Print out the matrix created
print(f"""The matrix for the production units per region and flavor type is:
{pr_matrix}""")


The matrix for the production units per region and flavor type is:
[[0 2.0*X_2 + 0.0 2.0*X_3 + 0.0 1.0*X_4 + 0.0 2.0*X_5 + 0.0 1.0*X_6 + 0.0]
 [1.0*X_1 + 0.0 2.0*X_2 + 0.0 1.0*X_3 + 0.0 1.0*X_4 + 0.0 1.0*X_5 + 0.0 0]]


In [6]:
# The maximum of the production runs is 5
# Add this to the model
model += p.lpSum(pr_matrix) <= 5

# At least one port needs to be chosen
# The ports are linked to region 1, 2, 4 and 5
# Add this to the model
model += p.lpSum(allocation[ : , [0,1,3,5] ]) >= 1


# We considered which regions are not possible if three regions are selected by the model
# This is because they need to be adjacent to each other
# The combinations below were identified

model += p.lpSum(allocation[ : , [0,4,5] ]) <= 2
model += p.lpSum(allocation[ : , [0,2,4] ]) <= 2
model += p.lpSum(allocation[ : , [0,2,5] ]) <= 2
model += p.lpSum(allocation[ : , [1,4,5] ]) <= 2
model += p.lpSum(allocation[ : , [2,0,5] ]) <= 2
model += p.lpSum(allocation[ : , [4,0,1] ]) <= 2
model += p.lpSum(allocation[ : , [5,0,1] ]) <= 2
model += p.lpSum(allocation[ : , [5,0,2] ]) <= 2
model += p.lpSum(allocation[ : , [5,1,2] ]) <= 2

In [7]:
# The model identified a combination of 2 regions
# This means that there is a further infeasible solution as regions are not adjacent
# As a result YRD (region 2) & Western China (region 6) cannot be chosen together and so was added as a constraint
model += p.lpSum(allocation[ : , [1,5] ]) <= 1


In [8]:
# Print out the model to ensure it is specified correctly
print(model)

Cornology-Problem:
MAXIMIZE
525000.0*X_1 + 3090000.0*X_2 + 1865000.0*X_3 + 1650000.0*X_4 + 2000000.0*X_5 + 1125000.0*X_6 + 0.0
SUBJECT TO
_C1: X_1 + 4 X_2 + 3 X_3 + 2 X_4 + 3 X_5 + X_6 <= 5

_C2: X_1 + X_2 + X_4 + X_6 >= 1

_C3: X_1 + X_5 + X_6 <= 2

_C4: X_1 + X_3 + X_5 <= 2

_C5: X_1 + X_3 + X_6 <= 2

_C6: X_2 + X_5 + X_6 <= 2

_C7: X_1 + X_3 + X_6 <= 2

_C8: X_1 + X_2 + X_5 <= 2

_C9: X_1 + X_2 + X_6 <= 2

_C10: X_1 + X_3 + X_6 <= 2

_C11: X_2 + X_3 + X_6 <= 2

_C12: X_2 + X_6 <= 1

VARIABLES
0 <= X_1 <= 1 Integer
0 <= X_2 <= 1 Integer
0 <= X_3 <= 1 Integer
0 <= X_4 <= 1 Integer
0 <= X_5 <= 1 Integer
0 <= X_6 <= 1 Integer



## Solve the Model

In [9]:
# The function to solve the model
model.solve(solver = None)

print(f"""The model finds the following result: {p.LpStatus[model.status]}""")


The model finds the following result: Optimal


## Optimal Solution

In [10]:
# Print the optimal value of the objective function
print("Maximized Profit:", model.objective.value())

# Print the chosen regions, 1 as chosen, 0 as non chosen
# Identify if there's any errors

for dec_var in model.variables():
    try:
        print(dec_var.name,"=", dec_var.value())
    except:
        print("error couldnt find value")

Maximized Profit: 3650000.0
X_1 = 0.0
X_2 = 0.0
X_3 = 0.0
X_4 = 1.0
X_5 = 1.0
X_6 = 0.0


##### Comment:
This means that the North Central and South Central region is selected to reach a maximized profit of $3,650,000. The port would be in North Central as there is no option for a port in the South Central region.

## Alternative Solution (Second Most Profit)

In [11]:
# The model to date was copied into another name model_alt
# In order to identify the next best solution a further constraint was added
# Exclude the combination of North Central & South Central

model_alt = model.copy()

model_alt += p.lpSum(allocation[ : , [3,4] ]) <= 1
print(model)
print(model_alt)

Cornology-Problem:
MAXIMIZE
525000.0*X_1 + 3090000.0*X_2 + 1865000.0*X_3 + 1650000.0*X_4 + 2000000.0*X_5 + 1125000.0*X_6 + 0.0
SUBJECT TO
_C1: X_1 + 4 X_2 + 3 X_3 + 2 X_4 + 3 X_5 + X_6 <= 5

_C2: X_1 + X_2 + X_4 + X_6 >= 1

_C3: X_1 + X_5 + X_6 <= 2

_C4: X_1 + X_3 + X_5 <= 2

_C5: X_1 + X_3 + X_6 <= 2

_C6: X_2 + X_5 + X_6 <= 2

_C7: X_1 + X_3 + X_6 <= 2

_C8: X_1 + X_2 + X_5 <= 2

_C9: X_1 + X_2 + X_6 <= 2

_C10: X_1 + X_3 + X_6 <= 2

_C11: X_2 + X_3 + X_6 <= 2

_C12: X_2 + X_6 <= 1

VARIABLES
0 <= X_1 <= 1 Integer
0 <= X_2 <= 1 Integer
0 <= X_3 <= 1 Integer
0 <= X_4 <= 1 Integer
0 <= X_5 <= 1 Integer
0 <= X_6 <= 1 Integer

Cornology-Problem:
MAXIMIZE
525000.0*X_1 + 3090000.0*X_2 + 1865000.0*X_3 + 1650000.0*X_4 + 2000000.0*X_5 + 1125000.0*X_6 + 0.0
SUBJECT TO
_C1: X_1 + 4 X_2 + 3 X_3 + 2 X_4 + 3 X_5 + X_6 <= 5

_C2: X_1 + X_2 + X_4 + X_6 >= 1

_C3: X_1 + X_5 + X_6 <= 2

_C4: X_1 + X_3 + X_5 <= 2

_C5: X_1 + X_3 + X_6 <= 2

_C6: X_2 + X_5 + X_6 <= 2

_C7: X_1 + X_3 + X_6 <= 2

_C8: X_

In [12]:
# Solve the alternative model
model_alt.solve(solver = None)

print(f"""The model finds the following result: {p.LpStatus[model_alt.status]}""")


The model finds the following result: Optimal


In [13]:
# Print the optimal value of the objective function from the alternate model
print("Maximize Profit:", model_alt.objective.value())

# Print the chosen regions, 1 as chosen, 0 as non chosen
# Identify if there's any errors

for dec_var in model_alt.variables():
    try:
        print(dec_var.name,"=", dec_var.value())
    except:
        print("error couldnt find value")

Maximize Profit: 3615000.0
X_1 = 1.0
X_2 = 1.0
X_3 = 0.0
X_4 = 0.0
X_5 = 0.0
X_6 = 0.0


##### Comment:
This means that the alternative solution is to enter YRD and the North East to reach the profit of 3,615,000. Both of these are ports and therefore the choice of which port to use will depend on the benefit of placing a factory in those regions.